In [4]:
import streamlit as st 
from PyPDF2 import PdfReader 
from langchain.text_splitter import RecursiveCharacterTextSplitter 
import os 
from pydantic import BaseModel
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

import google.generativeai as genai 
from langchain.vectorstores import FAISS 
from langchain_google_genai import ChatGoogleGenerativeAI 
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate 
from dotenv import load_dotenv



In [5]:
load_dotenv()

False

In [7]:
genai.configure(api_key= "")

In [8]:
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text 

def get_text_from_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chubk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(texts = text_chunks, embedding = embeddings)
    vector_store.save_local("faiss_index")
    return vector_store

def get_conversation_chain(vector_store):
    Prompt_template = """ 
        Answer the question as detailed as possible form the given conext,dont try to hallucinate the answers,
        question: {question}\n
        context : {context}
        Answer : 
        """
    model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3)
    prompt = Prompt_template(template = PromptTemplate,input_variables=["question","context"])
    chain = load_qa_chain(model,chain_type="stuff",prompt=prompt)
    return chain

def user_input(useer_question):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    new_db = FAISS.load_local("faiss-index",embeddings)
    docs = new_db.similarity_search(useer_question)
    chain = get_conversation_chain()

    response = chain( 
        {"input_documents":docs,"question":useer_question},
        return_only_outputs=True
    )
    print(response)
    



